## Import and format json data
Importing playlists from jsons and formatting them into ratings data frame. (Get data from https://owncloud.tuwien.ac.at/index.php/s/A8Wx2TpFr0WznZh)

In [116]:
import pandas as pd
import numpy as np
import json

# load data
train = json.load(open('data/train.json', encoding = "utf8"))

dev = json.load(open('data/dev.json', encoding = "utf8"))
dev_trun = json.load(open('data/dev_trun.json', encoding="utf8"))

test = json.load(open('data/test.json', encoding = "utf8"))
test_trun = json.load(open('data/test_trun.json', encoding = "utf8"))

# create ratings df with training + dev for validation (equivalent to 'ratings_train' data frame in Assignment 4.1)
rating_list = []

seen = set()
track_list = []
for playlist in train['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
        if track['track_uri'] not in seen:
            seen.add(track['track_uri'])
            track_list.append([track['track_uri'], track['track_name'], track['artist_uri'], track['artist_name'], track['album_uri'], track['album_name'], track['duration_ms']])

for playlist in dev_trun['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
        if track['track_uri'] not in seen:
            seen.add(track['track_uri'])
            track_list.append([track['track_uri'], track['track_name'], track['artist_uri'], track['artist_name'], track['album_uri'], track['album_name'], track['duration_ms']])

ratings_train_dev = pd.DataFrame(rating_list, columns = ['pid', 'name', 'track_uri', 'rating'])
tracks = pd.DataFrame(track_list, columns = ['track_uri', 'track_name', 'artist_uri', 'artist_name', 'album_uri', 'album_name', 'duration_ms'])
                           
# create ratings df for testing
rating_list = []
for playlist in train['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
for playlist in test_trun['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])

ratings_train_test = pd.DataFrame(rating_list, columns = ['pid', 'name', 'track_uri', 'rating'])

# holdouts ('topN' equivalent)
dev_holdouts = {playlist['pid']:playlist['num_holdouts'] for playlist in dev['playlists']}
test_holdouts = {playlist['pid']:playlist['num_holdouts'] for playlist in test['playlists']}

#ratings_train_dev.head()
tracks.head()

# not sure if train and dev need to be combined.. train alone doesn't miss any tracks, 
# so we cannot test the model on it. Combining it with def, we can make predictions on
# missing tracks in dev playlists. EDIT: I think combining is right. Also combining train + test. 
# Otherwise, train isn't part of the model.


,track_uri,track_name,artist_uri,artist_name,album_uri,album_name,duration_ms
0,spotify:track:2CFGQXM3exBO8U9WGve2GY,Elle me dit,spotify:artist:5MmVJVhhYKQ86izuGHzJYA,MIKA,spotify:album:0zY3JRYHC0rycbFUqGcRpV,The Origin Of Love,216933
1,spotify:track:09TcIuH1ZO7i4vicWKoaN2,Papaoutai,spotify:artist:5j4HeCoUlzhfWtjAfM1acR,Stromae,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,racine carrée,232146
2,spotify:track:3Uyt0WO3wOopnUBCe9BaXl,Sur ma route,spotify:artist:7lMgpN1tEBQKpRoUMKB8iw,Black M,spotify:album:1D2Rs9qcENebbiDR5wk88T,Les yeux plus gros que le monde,252573
3,spotify:track:4h95OrjKWYzVIai1EkvZbt,Ta fête,spotify:artist:5j4HeCoUlzhfWtjAfM1acR,Stromae,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,racine carrée,175426
4,spotify:track:3pCN6X0566rmU6P5ZgFIYu,bâtard,spotify:artist:5j4HeCoUlzhfWtjAfM1acR,Stromae,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,racine carrée,208826


## Import and test recommenders

In [117]:
%load_ext autoreload
%autoreload 2
from Recommender_CB import ContentBasedRecommender
from Recommender_CF_UU import UUCFRecommender
from Recommender_MF import MFRecommender

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
## Matrix Fac.
mfr = MFRecommender()
mfr.build_model(ratings_train_dev)

recs = mfr.recommend('498917', topN = holdouts['498917'])
print(recs)


## UU-CB
uucf = UUCFRecommender()
uucf.build_model(ratings_train_dev)

recs = uucf.recommend('498917', topN = holdouts['498917'])
print(recs)

AttributeError: 'DataFrame' object has no attribute 'movieId'

In [ ]:
## Content-based 
cbr = ContentBasedRecommender('extended')
cbr.build_model(ratings_train_dev, tracks)

ok


In [136]:
recs = cbr.recommend(498917, topN = dev_holdouts.get(498917))
print(recs)

AttributeError: 'ContentBasedRecommender' object has no attribute 'playlist_profiles'

## Parameter tuning
Parameters are tuned through evaluating performance of different parameter settings on the train+dev (?) set. Those settings will be used for the test set.

In [ ]:
import metrics

#TODO: Parameter tuning

## Evaluation on test set 

In [ ]:
import metrics

#TODO: evaluation on test set